# 元编程

## 9.1 在函数上添加包装器

你想在函数上添加一个包装器，增加额外的操作处理 (比如日志、计时等)

如果你想使用额外的代码包装一个函数，可以定义一个装饰器函数

In [5]:
import time
from functools import wraps

def timethis(func):
    '''
    Decorator that reports the execution time
    '''
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print('Timer function name is {}, cost time {}'.format(func.__name__, end-start))
        return result
    return wrapper

In [6]:
@timethis
def countdown(n):
    while n > 0:
        n -= 1
countdown(100000)

Timer function name is countdown, cost time 0.005001068115234375


In [7]:
countdown(100000)

Timer function name is countdown, cost time 0.004999637603759766


一个装饰器就是一个函数，它接受一个函数作为参数并返回一个新的函数。当你像下面这样写

In [8]:
@timethis
def countdown(n):
    pass

# equal
def countdown(n):
    pass
countdown = timethis(countdown)

顺便说一下，内置的装饰器比如@staticmethod,@classmethod,@property 
原理也是一样的。例如，下面这两个代码片段是等价的

In [9]:
class A:
    @classmethod
    def method(cls):
        pass
    
class B:
    def method(cls):
        pass
    method = classmethod(method)

在上面的 wrapper() 函数中，装饰器内部定义了一个使用 *args 和 **kwargs 来接受任意参数的函数。在这个函数里面调用了原始函数并将其结果返回，不过你还可以添加其他额外的代码 (比如计时)。然后这个新的函数包装器被作为结果返回来代替原始函数。

需要强调的是装饰器并不会修改原始函数的参数签名以及返回值。使用 *args和 **kwargs 目的就是确保任何参数都能适用。而返回结果值基本都是调用原始函数func(*args, **kwargs) 的返回结果，其中 func 就是原始函数。

刚开始学习装饰器的时候，会使用一些简单的例子来说明，比如上面演示的这个。不过实际场景使用时，还是有一些细节问题要注意的。比如上面使用 @wraps(func) 注解是很重要的，它能保留原始函数的元数据 (下一小节会讲到)，新手经常会忽略这个细节。接下来的几个小节我们会更加深入的讲解装饰器函数的细节问题，如果你想构造你自己的装饰器函数，需要认真看一下。

## 9.2 创建装饰器时保留函数元信息

你写了一个装饰器作用在某个函数上，但是这个函数的重要的元信息比如名字、文档字符串、注解和参数签名都丢失了

任何时候你定义装饰器的时候，都应该使用 functools 库中的 @wraps 装饰器来注解底层包装函数

In [10]:
import time
from functools import wraps

def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

下面我们使用这个被包装后的函数并检查它的元信息

In [11]:
# 元信息 ?
# 参数签名信息？

@timethis
def countdown(n):
    while n > 0:
        n -= 1

In [12]:
countdown(100000)

countdown 0.005000591278076172


In [13]:
countdown.__name__

'countdown'

In [14]:
countdown.__doc__

In [15]:
countdown.__annotations__

{}

在编写装饰器的时候复制元信息是一个非常重要的部分。如果你忘记了使用 @wraps
，那么你会发现被装饰函数丢失了所有有用的信息。比如如果忽略 @wraps 后的效果是
下面这样的：

In [16]:
# 上面的代码没忘记啊，但还是丢失了所有有用的信息

@wraps 有一个重要特征是它能让你通过属性 __wrapped__ 直接访问被包装函数

In [17]:
countdown.__wrapped__(100000)

__wrapped__ 属性还能让被装饰函数正确暴露底层的参数签名信息。例如：

In [18]:
from inspect import signature
print(signature(countdown))

(n)


一个很普遍的问题是怎样让装饰器去直接复制原始函数的参数签名信息，如果想
自己手动实现的话需要做大量的工作，最好就简单的使用 @wraps 装饰器。通过底层的
__wrapped__ 属性访问到函数签名信息

## 9.3 解除一个装饰器

一个装饰器已经作用在一个函数上，你想撤销它，直接访问原始的未包装的那个函
数。

假设装饰器是通过 @wraps (参考 9.2 小节) 来实现的，那么你可以通过访问
__wrapped__ 属性来访问原始函数

In [19]:
@timethis
def add(x, y):
    return x + y


In [20]:
orig_add = add.__wrapped__
orig_add(3, 4)

7

直接访问未包装的原始函数在调试、内省和其他函数操作时是很有用的。但是我
们这里的方案仅仅适用于在包装器中正确使用了 @wraps 或者直接设置了 __wrapped__
属性的情况。

如果有多个包装器，那么访问 __wrapped__ 属性的行为是不可预知的，应该避免
这样做。在 Python3.3 中，它会略过所有的包装层，比如，假如你有如下的代码：

In [21]:
from functools import wraps

def decorator1(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Decorator 1')
        return func(*args, **kwargs)
    return wrapper

def decorator2(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Decorator 2')
        return func(*args, **kwargs)
    return wrapper

@decorator1
@decorator2
def add(x, y):
    return x + y

In [22]:
add(2, 3)

Decorator 1
Decorator 2


5

In [23]:
add.__wrapped__(2, 3)

Decorator 2


5

最后要说的是，并不是所有的装饰器都使用了 @wraps ，因此这里的方案并不全部
适用。

特别的，内置的装饰器 @staticmethod 和 @classmethod 就没有遵循这个约定
(它们把原始函数存储在属性 __func__ 中)

## 9.4 定义一个带参数的装饰器

一个可以接受参数的装饰器

我们用一个例子详细阐述下接受参数的处理过程。假设你想写一个装饰器，给函数
添加日志功能，同时允许用户指定日志的级别和其他的选项。下面是这个装饰器的定义
和使用示例

In [24]:
from functools import wraps

import logging

def logged(level, name=None, message=None):
    """Add logging to a function
    
    level:the logging level
    name:the logger name
    message:the log message
    
    if name and message aren't specified,default to the function's module and name
    """
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
        return wrapper
    return decorate

In [25]:
@logged(logging.DEBUG)
def add(x, y):
    return x + y

In [26]:
@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

初看起来，这种实现看上去很复杂，但是核心思想很简单。最外层的函数 logged()
接受参数并将它们作用在内部的装饰器函数上面。内层的函数 decorate() 接受一个函
数作为参数，然后在函数上面放置一个包装器。这里的关键点是包装器是可以使用传递
给 logged() 的参数的

定义一个接受参数的包装器看上去比较复杂主要是因为底层的调用序列。特别的，
如果你有下面这个代码

In [27]:
@decorator(x, y, z)
def func(a, b):
    pass
    
# equal
def func(a, b):
    pass
func = decorator(x, y, z)(func)

NameError: name 'decorator' is not defined

decorator(x, y, z) 的返回结果必须是一个可调用对象，它接受一个函数作为参
数并包装它，可以参考 9.7 小节中另外一个可接受参数的包装器例子

## 9.5 可自定义属性的装饰器

你想写一个装饰器来包装一个函数，并且允许用户提供参数在运行时控制装饰器
行为

引入一个访问函数，使用 nonlocal 来修改内部变量。然后这个访问函数被作为一
个属性赋值给包装函数

In [38]:
# 访问函数

from functools import wraps, partial

import logging

def attach_wrapper(obj, func=None):
    if func is None:
        return partial(attach_wrapper, obj)
    setattr(obj, func.__name__, func)
    return func

def logged(level, name=None, message=None):
    """Add logging to a function
    
    level:the logging level
    name:the logger name
    message:the log message
    
    if name and message aren't specified,default to the function's module and name
    """
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
        
        @attach_wrapper(wrapper)
        def set_level(newlevel):
            nonlocal level
            level = newlevel
            
        @attach_wrapper(wrapper)
        def set_message(newmsg):
            nonlocal logmsg
            logmsg = newmsg
            
        return wrapper
    return decorate

In [ ]:
# use example
@logged(logging.DEBUG)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)
add(2, 3)

In [ ]:
add.set_message('Add called')
add(2, 3)

In [ ]:
add.set_level(logging.WARNING)
add(2, 3)

这一小节的关键点在于访问函数 (如 set_message() 和 set_level() )，它们被作
为属性赋给包装器。每个访问函数允许使用 nonlocal 来修改函数内部的变量。

还有一个令人吃惊的地方是访问函数会在多层装饰器间传播 (如果你的装饰器都使
用了 @functools.wraps 注解)。例如，假设你引入另外一个装饰器，比如 9.2 小节中的
@timethis ，像下面这样

......

In [33]:
@timethis
@logged(logging.DEBUG)
def countdown(n):
    while n > 0:
        n -= 1

In [34]:
countdown(10000000)

countdown 0.606137752532959


In [35]:
countdown.set_level(logging.WARNING)

In [36]:
countdown.set_message('Counting down to zero')
countdown(100000)

Counting down to zero


countdown 0.006001472473144531


In [39]:
@timethis
@logged(logging.DEBUG)
def countdown(n):
    while n > 0:
        n -= 1
        
@attach_wrapper(wrapper)
def get_level():
    return level

wrapper.get_level = lambda : level

NameError: name 'wrapper' is not defined

一个比较难理解的地方就是对于访问函数的首次使用。例如，你可能会考虑另外一
个方法直接访问函数的属性

In [ ]:
@wraps(func)
def wrapper(*args, **kwargs):
    wrapper.log.log(wrapper.level, wrapper.logmsg)
    return func(*args, **kwargs)

wrapper.level = level
wrapper.logmsg = logmsg
wrapper.log = log

这个方法也可能正常工作，但前提是它必须是最外层的装饰器才行。如果它的上面
还有另外的装饰器 (比如上面提到的 @timethis 例子)，那么它会隐藏底层属性，使得
修改它们没有任何作用。而通过使用访问函数就能避免这样的局限性

## 带可选参数的装饰器

你想写一个装饰器，既可以不传参数给它，比如 @decorator ，也可以传递可选参
数给它，比如 @decorator(x,y,z)

下面是 9.5 小节中日志装饰器的一个修改版本

In [1]:
from functools import wraps, partial

import logging

def logged(func=None, *, level=logging.DEBUG, name=None, message=None):
    if func is None:
        return partial(logged, level=level, name=name, message=message)
    
    logname = name if name else func.__module__
    log = logging.getLogger(logname)
    logmsg = message if message else func.__name__
    
    @wraps(func) # ?
    def wrapper(*args, **kwargs):
        log.log(level, logmsg)
        return func(*args, **kwargs)
    return wrapper

In [2]:
# example use
# 可以看到，@logged 装饰器可以同时不带参数或带参数
@logged
def add(x, y):
    return x + y

In [3]:
@logged(level=logging.CRITICAL, name='example')
def spam():
    print('Spam!')
    

这里提到的这个问题就是通常所说的编程一致性问题。当我们使用装饰器的时候，
大部分程序员习惯了要么不给它们传递任何参数，要么给它们传递确切参数。其实从技
术上来讲，我们可以定义一个所有参数都是可选的装饰器，就像下面这样

In [4]:
@logged()
def add(x, y):
    return x + y

但是，这种写法并不符合我们的习惯，有时候程序员忘记加上后面的括号会导致错误。这里我们向你展示了如何以一致的编程风格来同时满足没有括号和有括号两种情
况。

为了理解代码是如何工作的，你需要非常熟悉装饰器是如何作用到函数上以及它们的调用规则。对于一个像下面这样的简单装饰器

In [5]:
#use
@logged
def add(x, y):
    return x + y

# as the same
def add(x, y):
    return x + y
add = logged(add) # 参数如何传递下去

In [7]:
add

<function __main__.add(x, y)>

这时候，被装饰函数会被当做第一个参数直接传递给 logged 装饰器。因此，
logged() 中的第一个参数就是被包装函数本身。所有其他参数都必须有默认值。
而对于一个下面这样有参数的装饰器：

In [8]:
@logged(level=logging.CRITICAL, name = 'example')
def spam():
    print('Spam')

In [9]:
# as the same
def spam():
    print('Spam')

spam = logged(level=logging.CRITICAL, name= 'example')(spam)

初始调用 logged() 函数时，被包装函数并没有传递进来。因此在装饰器内，它必
须是可选的。这个反过来会迫使其他参数必须使用关键字来指定。并且，但这些参数被
传递进来后，装饰器要返回一个接受一个函数参数并包装它的函数 (参考 9.5 小节)。为
了这样做，我们使用了一个技巧，就是利用 functools.partial 。它会返回一个未完
全初始化的自身，除了被包装函数外其他参数都已经确定下来了。可以参考 7.8 小节获
取更多 partial() 方法的知识。

## 利用装饰器强制函数上的类型检查

作为某种编程规约，你想在对函数参数进行强制类型检查。

在演示实际代码前，先说明我们的目标：能对函数参数类型进行断言，类似下面这
样：

In [12]:
from inspect import signature
from functools import wraps

def typeassert(*ty_args, **ty_kwargs):
    def decorate(func):
        if not __debug__: # ? 
            return func
        
        sig = signature(func)
        bound_type = sig.bind_partial(*ty_args, **ty_kwargs).arguments
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            bound_value = sig.bind(*args, **kwargs)
            for name, value in bound_value.arguments.items():
                if name in bound_type:
                    if not isinstance(value, bound_type[name]):
                        raise TypeError('Argument {} must be {}'.format(name, bound_type[name]))
            return func(*args, **kwargs)
        return wrapper
    return decorate

In [14]:
@typeassert(int, z=int)
def spam(x, y, z= 42):
    print(x, y, z)
    
spam(1, 2, 3)

1 2 3


In [19]:
@typeassert(str,z=float)
def myself(x,y,z=1.0):
    print(x, y, z)
myself('liu', 'gang', 5.1)

liu gang 5.1


In [20]:
myself(1, 2, 'str')

TypeError: Argument x must be <class 'str'>

装饰器只会在函数定义时被调用一次。有时候你去掉装饰器的功能，那么你
只需要简单的返回被装饰函数即可。下面的代码中，如果全局变量　 __debug__ 被设
置成了 False(当你使用-O 或-OO 参数的优化模式执行程序时)，那么就直接返回未修改
过的函数本身：

其次，这里还对被包装函数的参数签名进行了检查，我们使用了 inspect.
signature() 函数。简单来讲，它运行你提取一个可调用对象的参数签名信息。例
如：

In [21]:
from inspect import signature
def spam(x, y, z=42):
    pass

sig = signature(spam)
print(sig)

(x, y, z=42)


In [22]:
sig.parameters

mappingproxy({'x': <Parameter "x">,
              'y': <Parameter "y">,
              'z': <Parameter "z=42">})

In [23]:
sig.parameters['z'].name

'z'

In [24]:
sig.parameters['z'].default

42

In [25]:
sig.parameters['z'].kind

<_ParameterKind.POSITIONAL_OR_KEYWORD: 1>

In [26]:
sig.parameters['x'].kind

<_ParameterKind.POSITIONAL_OR_KEYWORD: 1>

In [27]:
sig.parameters['y'].kind

<_ParameterKind.POSITIONAL_OR_KEYWORD: 1>

装饰器的开始部分，我们使用了 bind_partial() 方法来执行从指定类型到名称的
部分绑定。下面是例子演示

In [28]:
bound_types = sig.bind_partial(int, z=int)
bound_types

<BoundArguments (x=<class 'int'>, z=<class 'int'>)>

在这个部分绑定中，你可以注意到缺失的参数被忽略了 (比如并没有对 y 进行绑定)。不过最重要的是创建了一个有序字典 bound_types.arguments 这个字典会将参数名以函数签名中相同顺序映射到指定的类型值上面去。

在我们的装饰器例子中，这个映射包含了我们要强制指定的类型断言。
在 装 饰 器 创 建 的 实 际 包 装 函 数 中 使 用 到 了 sig.bind() 方 法。bind() 跟
bind_partial() 类似，但是它不允许忽略任何参数。因此有了下面的结果

In [29]:
bound_values= sig.bind(1, 2, 3,)
bound_values.arguments

OrderedDict([('x', 1), ('y', 2), ('z', 3)])

使用这个映射我们可以很轻松的实现我们的强制类型检查：

In [30]:
for name, value in bound_values.arguments.items():
    if name in bound_types.arguments:
        if not isinstance(value, bound_types.arguments[name]):
            raise TypeError()

不过这个方案还有点小瑕疵，它对于有默认值的参数并不适用。比如下面的代码可
以正常工作，尽管 items 的类型是错误的

In [35]:
@typeassert(int, list)
def bar(x, items=None):
    if items is None:
        items = []
    items.append(x)
    return items
bar(2, 3)

TypeError: Argument items must be <class 'list'>

In [36]:
bar(2)

[2]

In [37]:
bar(2,[1,3])

[1, 3, 2]

最后一点是关于适用装饰器参数和函数注解之间的争论。例如，为什么不像下面这
样写一个装饰器来查找函数中的注解呢

In [38]:
@typeassert
def spam(x:int, y, z:int=42):
    print(x, y, z)

一个可能的原因是如果使用了函数参数注解，那么就被限制了。如果注解被用来做
类型检查就不能做其他事情了。而且 @typeassert 不能再用于使用注解做其他事情的
函数了。而使用上面的装饰器参数灵活性大多了，也更加通用。

## 将装饰器定义为类的一部分

你想在类中定义装饰器，并将其作用在其他函数或方法上

在类里面定义装饰器很简单，但是你首先要确认它的使用方式。比如到底是作为一
个实例方法还是类方法。下面我们用例子来阐述它们的不同

In [1]:
from functools import wraps

class A:
    # Decorator as an instance method
    def decorator1(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('Decorator 1')
            return func(*args, **kwargs)
        return wrapper
    
    # Decorator as  a class method
    @classmethod
    def decorator2(cls, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('Decorator 2')
            return func(*args, **kwargs)
        return wrapper

In [2]:
# As an instance method
a = A()
@a.decorator1
def spam():
    pass

@A.decorator2
def grok():
    pass

TypeError: decorator2() missing 1 required positional argument: 'func'

在类中定义装饰器初看上去好像很奇怪，但是在标准库中有很多这样的例子。特别
的，@property 装饰器实际上是一个类，它里面定义了三个方法 getter(), setter(),
deleter() , 每一个方法都是一个装饰器。例如：

In [3]:
class Person():
    first_name = property()
    @first_name.getter
    def first_name(self):
        return self._first_name
    
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value

它为什么要这么定义的主要原因是各种不同的装饰器方法会在关联的 property 实例上操作它的状态。因此，任何时候只要你碰到需要在装饰器中记录或绑定信息，那么这不失为一种可行方法。

在类中定义装饰器有个难理解的地方就是对于额外参数 self 或 cls 的正确使用。尽管最外层的装饰器函数比如 decorator1() 或 decorator2() 需要提供一个 self 或cls 参数，但是在两个装饰器内部被创建的 wrapper() 函数并不需要包含这个 self 参数。你唯一需要这个参数是在你确实要访问包装器中这个实例的某些部分的时候。其他情况下都不用去管它。

对于类里面定义的包装器还有一点比较难理解，就是在涉及到继承的时候。例如，假设你想让在 A 中定义的装饰器作用在子类 B 中。你需要像下面这样写：

In [ ]:
class B(A):
    @A.decorator2
    def bar(self):
        pass
#装饰器要被定义成类方法并且你必须显式的使用父类名去调用它。
#你不能使用 @B.decorator2 ，因为在方法定义时，这个类 B 还没有被创建

## 将装饰器定义为类